# Quantitative Assessment of Surgical Skills based on Muscular Activities

In [1]:
import torch
import numpy as np 
import scipy.io as sio
from os.path import dirname, join as pjoin
import matplotlib.pyplot as plt
from sklearn import preprocessing

### importing and formating the data from matlab : 

In [2]:
def exe_to_global_label(restimulus, inden):
    """
    this will take the label of ex2  or ex3 and convert it to a global label system
    the global label system is uniform for ex1,2 and 3 and goes from 0 to 52 
    """
    
    for i in range(len(restimulus)):
        if restimulus[i] != 0:
            restimulus[i] += inden 
        else:
            restimulus[i]=0
    return restimulus 


In [3]:
def get_data(n):
   
    """
    return the emg data and corresponding restimulus (for the 3 exos) of the n-th person
    """
    data_dir = pjoin('db_1', 'data', f's{n}')
    
    file_name_e1 = pjoin(data_dir, f'S{n}_A1_E1.mat')
    file_name_e2 = pjoin(data_dir, f'S{n}_A1_E2.mat')
    file_name_e3 = pjoin(data_dir, f'S{n}_A1_E3.mat')
    
    contents_e1 = sio.loadmat(file_name_e1)
    contents_e2 = sio.loadmat(file_name_e2)
    contents_e3 = sio.loadmat(file_name_e3)
    
    # those are np.array 
    emg_1 = contents_e1['emg']
    emg_2 = contents_e2['emg']
    emg_3 = contents_e3['emg']

    lbl_1 = contents_e1['restimulus']
    lbl_2 = contents_e2['restimulus']
    lbl_3 = contents_e3['restimulus']

    lbl_2 = exe_to_global_label(lbl_2,12) # so transforming the label into 13-29 (as 1-12 are already taken by ex_1)
    lbl_3 = exe_to_global_label(lbl_3,29) # so transforming the label into 30-52 (as 1-29are already taken by ex_1 and ex_2)  

    emg = np.concatenate((emg_1,emg_2,emg_3))
    labels = np.concatenate((lbl_1,lbl_2,lbl_3))
    
    return emg ,labels 

In [4]:
# concatenating the datas from all the subjects 
nb_subjects = 10 
emg= np.empty((0,10))
labels = np.empty((0,1))
for i in range(1,nb_subjects+1):
    emg_sub , labels_sub = get_data(i)
    emg = np.concatenate((emg,emg_sub))
    labels = np.concatenate((labels,labels_sub))
    
print(emg.shape)
print(labels.shape)

nb_channel = 10 

(4612696, 10)
(4612696, 1)


### separating the data by repetition : 
Think about the fact that the rest state will way more present than the other ones. Do we need to do something about it during training ? 

In [5]:
class repet():
    def __init__(self,emg,label):
        self.emg = emg
        self.label = label
        
        
class segment():
    
    def __init__(self,emg,label,nb_channel= 10):
        self.emg = emg
        self.label = label 
        self.nb_channel= nb_channel
        
    def mav(self):
        emg=self.emg
        nb_channel = self.nb_channel
        emg = np.absolute(emg)
        n  = emg.shape[0]
        mav=np.empty((1,nb_channel))
        for i in range(nb_channel):# for each channel
            mav[0][i] = (1/n)* np.sum(emg[i],axis = 0)
        return mav  
   
   
    def var(self):
        emg=self.emg
        nb_channel = self.nb_channel
        var = np.empty((1,nb_channel))
        for i in range(0,nb_channel):
            var[0][i]=np.var(emg[i])
        return var
    
    def wl(self):
        emg=self.emg
        nb_channel = self.nb_channel
        wl = np.empty((1,nb_channel))
        for i in range(0,nb_channel):
            wl[0][i]= sum([abs(emg[i][j] - emg[i][j-1]) for j in range(1, len(emg[i]))])
        return wl
    


    def get_features_by_channel(self,channel_nb):
        #return the all the features of this segment (for a given channel)
        feature_list = np.empty((0))
        
        feature_list = np.append(feature_list,self.mav()[0][channel_nb])
        feature_list = np.append(feature_list,self.var()[0][channel_nb])
        feature_list = np.append(feature_list,self.wl()[0][channel_nb])
        
        return feature_list

In [6]:
def separate_repetition(emg,labels):
   
    """
    we have 53 possible class (52 movements and 1 for the rest state)
    
    input : - emg data of all the participant concatenated 
            - restimulus data , so the label corresponding to the emg data 
            
    output : - a 1-D list of repet elements
    """
    
    nb_samples = len(labels)
    
    print("Number of samples : " , nb_samples)
    list_repet = np.empty((0))
    
    accum = 0 
    current_lbl = 0 
    for i in range(nb_samples):
        if current_lbl == labels[i]:
            accum += 1
        else :
            diff = i - accum
            new_rep = repet(emg[diff:i],labels[i])
            
            list_repet = np.append(list_repet, new_rep)
            
            current_lbl = labels[i]
            accum = 0 
    return list_repet

In [7]:
list_repetition = separate_repetition(emg,labels)
print(list_repetition.shape)



Number of samples :  4612696
(10400,)


In [8]:
def segmentation(list_repet, window_type = 'adjacent', length = 256):
   
    """
    input : - the list a the emg data separated by repetition 
            - a param window_type (either adjacent or overlapped)(see "Myoelectric control systems— A survey")
            - the length of each window
    output : - a list of all the segments for all the exercice and all subjects
    """
    nb_repet = len(list_repet)
    print("We have " , nb_repet , "repetitions of excercices in the data")
    
    seg_data = np.empty((0))
    
    if window_type == 'adjacent': 
        
        for i in range(nb_repet):
            nb_windows = list_repet[i].emg.shape[0] // length #integer division
           
            for j in range(nb_windows): 
                emg_seg = list_repet[i].emg[j*length:(j+1)*length]
                label_seg = list_repet[i].label
                
                new_seg = segment(emg_seg,label_seg)
                seg_data = np.append(seg_data,new_seg) 
           
    if window_type == 'overlapped':
        print("Not implemented yet")
    print("Data shape : ", seg_data.shape)
    print(seg_data.shape)
    return seg_data
    

In [9]:
list_seg = segmentation(list_repetition)
print("shape : ", list_seg.shape)
print(list_seg[0].get_features_by_channel(9))

We have  10400 repetitions of excercices in the data
Data shape :  (12688,)
(12688,)
shape :  (12688,)
[0.0003707  0.00020374 0.1027    ]


# Classification part : 

In [10]:
from sklearn import svm
from sklearn.model_selection import train_test_split

In [11]:
nb_feature = 3

input_feature = np.empty((nb_channel,list_seg.shape[0],nb_feature))

l = list_seg.shape[0]
for i in range(0,nb_channel):
    print(i)
    for j in range(0,list_seg.shape[0]):
        print(j,"/",l)
        input_feature[i][j] = list_seg[j].get_features_by_channel(i)

0
0 / 12688
1 / 12688
2 / 12688
3 / 12688
4 / 12688
5 / 12688
6 / 12688
7 / 12688
8 / 12688
9 / 12688
10 / 12688
11 / 12688
12 / 12688
13 / 12688
14 / 12688
15 / 12688
16 / 12688
17 / 12688
18 / 12688
19 / 12688
20 / 12688
21 / 12688
22 / 12688
23 / 12688
24 / 12688
25 / 12688
26 / 12688
27 / 12688
28 / 12688
29 / 12688
30 / 12688
31 / 12688
32 / 12688
33 / 12688
34 / 12688
35 / 12688
36 / 12688
37 / 12688
38 / 12688
39 / 12688
40 / 12688
41 / 12688
42 / 12688
43 / 12688
44 / 12688
45 / 12688
46 / 12688
47 / 12688
48 / 12688
49 / 12688
50 / 12688
51 / 12688
52 / 12688
53 / 12688
54 / 12688
55 / 12688
56 / 12688
57 / 12688
58 / 12688
59 / 12688
60 / 12688
61 / 12688
62 / 12688
63 / 12688
64 / 12688
65 / 12688
66 / 12688
67 / 12688
68 / 12688
69 / 12688
70 / 12688
71 / 12688
72 / 12688
73 / 12688
74 / 12688
75 / 12688
76 / 12688
77 / 12688
78 / 12688
79 / 12688
80 / 12688
81 / 12688
82 / 12688
83 / 12688
84 / 12688
85 / 12688
86 / 12688
87 / 12688
88 / 12688
89 / 12688
90 / 12688
91 / 12

798 / 12688
799 / 12688
800 / 12688
801 / 12688
802 / 12688
803 / 12688
804 / 12688
805 / 12688
806 / 12688
807 / 12688
808 / 12688
809 / 12688
810 / 12688
811 / 12688
812 / 12688
813 / 12688
814 / 12688
815 / 12688
816 / 12688
817 / 12688
818 / 12688
819 / 12688
820 / 12688
821 / 12688
822 / 12688
823 / 12688
824 / 12688
825 / 12688
826 / 12688
827 / 12688
828 / 12688
829 / 12688
830 / 12688
831 / 12688
832 / 12688
833 / 12688
834 / 12688
835 / 12688
836 / 12688
837 / 12688
838 / 12688
839 / 12688
840 / 12688
841 / 12688
842 / 12688
843 / 12688
844 / 12688
845 / 12688
846 / 12688
847 / 12688
848 / 12688
849 / 12688
850 / 12688
851 / 12688
852 / 12688
853 / 12688
854 / 12688
855 / 12688
856 / 12688
857 / 12688
858 / 12688
859 / 12688
860 / 12688
861 / 12688
862 / 12688
863 / 12688
864 / 12688
865 / 12688
866 / 12688
867 / 12688
868 / 12688
869 / 12688
870 / 12688
871 / 12688
872 / 12688
873 / 12688
874 / 12688
875 / 12688
876 / 12688
877 / 12688
878 / 12688
879 / 12688
880 / 12688
881 

2378 / 12688
2379 / 12688
2380 / 12688
2381 / 12688
2382 / 12688
2383 / 12688
2384 / 12688
2385 / 12688
2386 / 12688
2387 / 12688
2388 / 12688
2389 / 12688
2390 / 12688
2391 / 12688
2392 / 12688
2393 / 12688
2394 / 12688
2395 / 12688
2396 / 12688
2397 / 12688
2398 / 12688
2399 / 12688
2400 / 12688
2401 / 12688
2402 / 12688
2403 / 12688
2404 / 12688
2405 / 12688
2406 / 12688
2407 / 12688
2408 / 12688
2409 / 12688
2410 / 12688
2411 / 12688
2412 / 12688
2413 / 12688
2414 / 12688
2415 / 12688
2416 / 12688
2417 / 12688
2418 / 12688
2419 / 12688
2420 / 12688
2421 / 12688
2422 / 12688
2423 / 12688
2424 / 12688
2425 / 12688
2426 / 12688
2427 / 12688
2428 / 12688
2429 / 12688
2430 / 12688
2431 / 12688
2432 / 12688
2433 / 12688
2434 / 12688
2435 / 12688
2436 / 12688
2437 / 12688
2438 / 12688
2439 / 12688
2440 / 12688
2441 / 12688
2442 / 12688
2443 / 12688
2444 / 12688
2445 / 12688
2446 / 12688
2447 / 12688
2448 / 12688
2449 / 12688
2450 / 12688
2451 / 12688
2452 / 12688
2453 / 12688
2454 / 12688

3411 / 12688
3412 / 12688
3413 / 12688
3414 / 12688
3415 / 12688
3416 / 12688
3417 / 12688
3418 / 12688
3419 / 12688
3420 / 12688
3421 / 12688
3422 / 12688
3423 / 12688
3424 / 12688
3425 / 12688
3426 / 12688
3427 / 12688
3428 / 12688
3429 / 12688
3430 / 12688
3431 / 12688
3432 / 12688
3433 / 12688
3434 / 12688
3435 / 12688
3436 / 12688
3437 / 12688
3438 / 12688
3439 / 12688
3440 / 12688
3441 / 12688
3442 / 12688
3443 / 12688
3444 / 12688
3445 / 12688
3446 / 12688
3447 / 12688
3448 / 12688
3449 / 12688
3450 / 12688
3451 / 12688
3452 / 12688
3453 / 12688
3454 / 12688
3455 / 12688
3456 / 12688
3457 / 12688
3458 / 12688
3459 / 12688
3460 / 12688
3461 / 12688
3462 / 12688
3463 / 12688
3464 / 12688
3465 / 12688
3466 / 12688
3467 / 12688
3468 / 12688
3469 / 12688
3470 / 12688
3471 / 12688
3472 / 12688
3473 / 12688
3474 / 12688
3475 / 12688
3476 / 12688
3477 / 12688
3478 / 12688
3479 / 12688
3480 / 12688
3481 / 12688
3482 / 12688
3483 / 12688
3484 / 12688
3485 / 12688
3486 / 12688
3487 / 12688

4435 / 12688
4436 / 12688
4437 / 12688
4438 / 12688
4439 / 12688
4440 / 12688
4441 / 12688
4442 / 12688
4443 / 12688
4444 / 12688
4445 / 12688
4446 / 12688
4447 / 12688
4448 / 12688
4449 / 12688
4450 / 12688
4451 / 12688
4452 / 12688
4453 / 12688
4454 / 12688
4455 / 12688
4456 / 12688
4457 / 12688
4458 / 12688
4459 / 12688
4460 / 12688
4461 / 12688
4462 / 12688
4463 / 12688
4464 / 12688
4465 / 12688
4466 / 12688
4467 / 12688
4468 / 12688
4469 / 12688
4470 / 12688
4471 / 12688
4472 / 12688
4473 / 12688
4474 / 12688
4475 / 12688
4476 / 12688
4477 / 12688
4478 / 12688
4479 / 12688
4480 / 12688
4481 / 12688
4482 / 12688
4483 / 12688
4484 / 12688
4485 / 12688
4486 / 12688
4487 / 12688
4488 / 12688
4489 / 12688
4490 / 12688
4491 / 12688
4492 / 12688
4493 / 12688
4494 / 12688
4495 / 12688
4496 / 12688
4497 / 12688
4498 / 12688
4499 / 12688
4500 / 12688
4501 / 12688
4502 / 12688
4503 / 12688
4504 / 12688
4505 / 12688
4506 / 12688
4507 / 12688
4508 / 12688
4509 / 12688
4510 / 12688
4511 / 12688

...

7907 / 12688
7908 / 12688
7909 / 12688
7910 / 12688
7911 / 12688
7912 / 12688
7913 / 12688
7914 / 12688
7915 / 12688
7916 / 12688
7917 / 12688
7918 / 12688
7919 / 12688
7920 / 12688
7921 / 12688
7922 / 12688
7923 / 12688
7924 / 12688
7925 / 12688
7926 / 12688
7927 / 12688
7928 / 12688
7929 / 12688
7930 / 12688
7931 / 12688
7932 / 12688
7933 / 12688
7934 / 12688
7935 / 12688
7936 / 12688
7937 / 12688
7938 / 12688
7939 / 12688
7940 / 12688
7941 / 12688
7942 / 12688
7943 / 12688
7944 / 12688
7945 / 12688
7946 / 12688
7947 / 12688
7948 / 12688
7949 / 12688
7950 / 12688
7951 / 12688
7952 / 12688
7953 / 12688
7954 / 12688
7955 / 12688
7956 / 12688
7957 / 12688
7958 / 12688
7959 / 12688
7960 / 12688
7961 / 12688
7962 / 12688
7963 / 12688
7964 / 12688
7965 / 12688
7966 / 12688
7967 / 12688
7968 / 12688
7969 / 12688
7970 / 12688
7971 / 12688
7972 / 12688
7973 / 12688
7974 / 12688
7975 / 12688
7976 / 12688
7977 / 12688
7978 / 12688
7979 / 12688
7980 / 12688
7981 / 12688
7982 / 12688
7983 / 12688

8901 / 12688
8902 / 12688
8903 / 12688
8904 / 12688
8905 / 12688
8906 / 12688
8907 / 12688
8908 / 12688
8909 / 12688
8910 / 12688
8911 / 12688
8912 / 12688
8913 / 12688
8914 / 12688
8915 / 12688
8916 / 12688
8917 / 12688
8918 / 12688
8919 / 12688
8920 / 12688
8921 / 12688
8922 / 12688
8923 / 12688
8924 / 12688
8925 / 12688
8926 / 12688
8927 / 12688
8928 / 12688
8929 / 12688
8930 / 12688
8931 / 12688
8932 / 12688
8933 / 12688
8934 / 12688
8935 / 12688
8936 / 12688
8937 / 12688
8938 / 12688
8939 / 12688
8940 / 12688
8941 / 12688
8942 / 12688
8943 / 12688
8944 / 12688
8945 / 12688
8946 / 12688
8947 / 12688
8948 / 12688
8949 / 12688
8950 / 12688
8951 / 12688
8952 / 12688
8953 / 12688
8954 / 12688
8955 / 12688
8956 / 12688
8957 / 12688
8958 / 12688
8959 / 12688
8960 / 12688
8961 / 12688
8962 / 12688
8963 / 12688
8964 / 12688
8965 / 12688
8966 / 12688
8967 / 12688
8968 / 12688
8969 / 12688
8970 / 12688
8971 / 12688
8972 / 12688
8973 / 12688
8974 / 12688
8975 / 12688
8976 / 12688
8977 / 12688

9884 / 12688
9885 / 12688
9886 / 12688
9887 / 12688
9888 / 12688
9889 / 12688
9890 / 12688
9891 / 12688
9892 / 12688
9893 / 12688
9894 / 12688
9895 / 12688
9896 / 12688
9897 / 12688
9898 / 12688
9899 / 12688
9900 / 12688
9901 / 12688
9902 / 12688
9903 / 12688
9904 / 12688
9905 / 12688
9906 / 12688
9907 / 12688
9908 / 12688
9909 / 12688
9910 / 12688
9911 / 12688
9912 / 12688
9913 / 12688
9914 / 12688
9915 / 12688
9916 / 12688
9917 / 12688
9918 / 12688
9919 / 12688
9920 / 12688
9921 / 12688
9922 / 12688
9923 / 12688
9924 / 12688
9925 / 12688
9926 / 12688
9927 / 12688
9928 / 12688
9929 / 12688
9930 / 12688
9931 / 12688
9932 / 12688
9933 / 12688
9934 / 12688
9935 / 12688
9936 / 12688
9937 / 12688
9938 / 12688
9939 / 12688
9940 / 12688
9941 / 12688
9942 / 12688
9943 / 12688
9944 / 12688
9945 / 12688
9946 / 12688
9947 / 12688
9948 / 12688
9949 / 12688
9950 / 12688
9951 / 12688
9952 / 12688
9953 / 12688
9954 / 12688
9955 / 12688
9956 / 12688
9957 / 12688
9958 / 12688
9959 / 12688
9960 / 12688

11075 / 12688
11076 / 12688
11077 / 12688
11078 / 12688
11079 / 12688
11080 / 12688
11081 / 12688
11082 / 12688
11083 / 12688
11084 / 12688
11085 / 12688
11086 / 12688
11087 / 12688
11088 / 12688
11089 / 12688
11090 / 12688
11091 / 12688
11092 / 12688
11093 / 12688
11094 / 12688
11095 / 12688
11096 / 12688
11097 / 12688
11098 / 12688
11099 / 12688
11100 / 12688
11101 / 12688
11102 / 12688
11103 / 12688
11104 / 12688
11105 / 12688
11106 / 12688
11107 / 12688
11108 / 12688
11109 / 12688
11110 / 12688
11111 / 12688
11112 / 12688
11113 / 12688
11114 / 12688
11115 / 12688
11116 / 12688
11117 / 12688
11118 / 12688
11119 / 12688
11120 / 12688
11121 / 12688
11122 / 12688
11123 / 12688
11124 / 12688
11125 / 12688
11126 / 12688
11127 / 12688
11128 / 12688
11129 / 12688
11130 / 12688
11131 / 12688
11132 / 12688
11133 / 12688
11134 / 12688
11135 / 12688
11136 / 12688
11137 / 12688
11138 / 12688
11139 / 12688
11140 / 12688
11141 / 12688
11142 / 12688
11143 / 12688
11144 / 12688
11145 / 12688
11146 

11929 / 12688
11930 / 12688
11931 / 12688
11932 / 12688
11933 / 12688
11934 / 12688
11935 / 12688
11936 / 12688
11937 / 12688
11938 / 12688
11939 / 12688
11940 / 12688
11941 / 12688
11942 / 12688
11943 / 12688
11944 / 12688
11945 / 12688
11946 / 12688
11947 / 12688
11948 / 12688
11949 / 12688
11950 / 12688
11951 / 12688
11952 / 12688
11953 / 12688
11954 / 12688
11955 / 12688
11956 / 12688
11957 / 12688
11958 / 12688
11959 / 12688
11960 / 12688
11961 / 12688
11962 / 12688
11963 / 12688
11964 / 12688
11965 / 12688
11966 / 12688
11967 / 12688
11968 / 12688
11969 / 12688
11970 / 12688
11971 / 12688
11972 / 12688
11973 / 12688
11974 / 12688
11975 / 12688
11976 / 12688
11977 / 12688
11978 / 12688
11979 / 12688
11980 / 12688
11981 / 12688
11982 / 12688
11983 / 12688
11984 / 12688
11985 / 12688
11986 / 12688
11987 / 12688
11988 / 12688
11989 / 12688
11990 / 12688
11991 / 12688
11992 / 12688
11993 / 12688
11994 / 12688
11995 / 12688
11996 / 12688
11997 / 12688
11998 / 12688
11999 / 12688
12000 

In [12]:
label = np.empty((nb_channel,list_seg.shape[0]))
print(list_seg[0].label[0])
for i in range(0,nb_channel):
    print(i)
    for j in range(0,list_seg.shape[0]):
        label[i][j] = list_seg[j].label[0]
        
print("shape",label.shape)
print(input_feature[0].shape)
print(list_seg[11000].label[0])
print(list_seg.shape[0])
print(label)
print(nb_channel)

1.0
0
1
2
3
4
5
6
7
8
9
shape (10, 12688)
(12688, 3)
35.0
12688
[[ 1.  0.  1. ... 52.  0.  0.]
 [ 1.  0.  1. ... 52.  0.  0.]
 [ 1.  0.  1. ... 52.  0.  0.]
 ...
 [ 1.  0.  1. ... 52.  0.  0.]
 [ 1.  0.  1. ... 52.  0.  0.]
 [ 1.  0.  1. ... 52.  0.  0.]]
10


In [13]:
input_feature_all =np.empty((0,nb_feature))
label_all=np.empty((0))
for i in range(0,nb_channel):
    input_feature_all = np.append(input_feature_all, input_feature[i], axis=0) #combination off all 10 chanels
    label_all = np.append(label_all,label[i])
print(input_feature_all.shape)
print(label_all.shape)

for i in range(0,nb_channel):
    input_feature_all[:][:][i] = preprocessing.normalize([input_feature_all[:][:][i]])

"""
x_train= np.empty((0,nb_feature))
y_train= np.empty((0))

x_test= np.empty((0,nb_feature))
y_test= np.empty((0))

for i in range(0,nb_channel): # in this part , we study all the channels togheter
    x_train_temp, x_test_temp, y_train_temp, y_test_temp = train_test_split(input_feature[i],label[i], test_size=0.1, 
                                                        shuffle=True ) 
    print(x_train_temp.shape)
    print(y_train_temp.shape)
    x_train= np.append(x_train,x_train_temp,axis=0)
    y_train= np.append(y_train,y_train_temp,axis=0)
    
    x_test= np.append(x_test,x_test_temp,axis=0)
    y_test= np.append(y_test,y_test_temp,axis=0)

print(x_train.shape)
"""
x_train_all, x_test_all, y_train_all, y_test_all = train_test_split(input_feature_all,label_all, test_size=0.2, 
                                                        shuffle=True )


(126880, 3)
(126880,)


In [14]:
a = np.array([1, 2])
b = np.array([4,5,6])
print(a.shape)

a = np.append(a,b)
print(a.shape)

(2,)
(5,)


In [15]:
"""
clf = svm.SVC(gamma='auto',kernel='rbf',class_weight='balanced',C=1.0, degree=3 ,max_iter= -1)
clf.fit(x_train_all, y_train_all)
"""

"\nclf = svm.SVC(gamma='auto',kernel='rbf',class_weight='balanced',C=1.0, degree=3 ,max_iter= -1)\nclf.fit(x_train_all, y_train_all)\n"

In [16]:
"""
compt = 0
for i in range(0,x_test_all.shape[0]):
    if clf.predict([x_test_all[i]])[0] - y_test_all[i] == 0.0:
        compt +=1
print(compt)

print(100*compt/x_test_all.shape[0],"% accuracy")
"""

'\ncompt = 0\nfor i in range(0,x_test_all.shape[0]):\n    if clf.predict([x_test_all[i]])[0] - y_test_all[i] == 0.0:\n        compt +=1\nprint(compt)\n\nprint(100*compt/x_test_all.shape[0],"% accuracy")\n'

In [17]:
"""
print(100*compt/x_test_all.shape[0],"% accuracy")
print(clf.class_weight_)
"""

'\nprint(100*compt/x_test_all.shape[0],"% accuracy")\nprint(clf.class_weight_)\n'

In [18]:
"""
#-------------------------------------------------------------
x_train, x_test, y_train, y_test = train_test_split(input_feature[0],label[0], test_size=0.2, 
                                                        shuffle=True )
                                                        """

'\n#-------------------------------------------------------------\nx_train, x_test, y_train, y_test = train_test_split(input_feature[0],label[0], test_size=0.2, \n                                                        shuffle=True )\n                                                        '

In [19]:
"""
clf = svm.SVC(gamma='auto',kernel='rbf',class_weight='balanced',C=1.0, degree=3 ,max_iter= 1)
clf.fit(x_train, y_train)
"""

"\nclf = svm.SVC(gamma='auto',kernel='rbf',class_weight='balanced',C=1.0, degree=3 ,max_iter= 1)\nclf.fit(x_train, y_train)\n"

In [20]:
"""
compt = 0
for i in range(0,x_test.shape[0]):
    if clf.predict([x_test[i]])[0] - y_test[i] == 0.0:
        compt +=1
print(compt)

print(100*compt/x_test_all.shape[0],"% accuracy")
"""

'\ncompt = 0\nfor i in range(0,x_test.shape[0]):\n    if clf.predict([x_test[i]])[0] - y_test[i] == 0.0:\n        compt +=1\nprint(compt)\n\nprint(100*compt/x_test_all.shape[0],"% accuracy")\n'

In [21]:
"""
clf_linear = svm.SVC(gamma='auto',kernel='linear',class_weight='balanced',C=1.0, degree=3 ,max_iter= -1)
clf_linear.fit(x_train_all, y_train_all)
"""

"\nclf_linear = svm.SVC(gamma='auto',kernel='linear',class_weight='balanced',C=1.0, degree=3 ,max_iter= -1)\nclf_linear.fit(x_train_all, y_train_all)\n"

In [22]:
"""
compt = 0
for i in range(0,x_test_all.shape[0]):
    if clf_linear.predict([x_test_all[i]])[0] - y_test_all[i] == 0.0:
        compt +=1
print(compt)

print(100*compt/x_test_all.shape[0],"% accuracy")
"""

'\ncompt = 0\nfor i in range(0,x_test_all.shape[0]):\n    if clf_linear.predict([x_test_all[i]])[0] - y_test_all[i] == 0.0:\n        compt +=1\nprint(compt)\n\nprint(100*compt/x_test_all.shape[0],"% accuracy")\n'

In [23]:
"""
clf_sig = svm.SVC(gamma='auto',kernel='sigmoid',class_weight='balanced',C=1.0,max_iter= -1)
clf_sig.fit(x_train_all, y_train_all)
"""

"\nclf_sig = svm.SVC(gamma='auto',kernel='sigmoid',class_weight='balanced',C=1.0,max_iter= -1)\nclf_sig.fit(x_train_all, y_train_all)\n"

In [24]:
"""
compt = 0
for i in range(0,x_test_all.shape[0]):
    if clf_sig.predict([x_test_all[i]])[0] - y_test_all[i] == 0.0:
        compt +=1
print(compt)

print(100*compt/x_test_all.shape[0],"% accuracy")
"""

'\ncompt = 0\nfor i in range(0,x_test_all.shape[0]):\n    if clf_sig.predict([x_test_all[i]])[0] - y_test_all[i] == 0.0:\n        compt +=1\nprint(compt)\n\nprint(100*compt/x_test_all.shape[0],"% accuracy")\n'

In [25]:
from sklearn.ensemble import RandomForestClassifier

In [26]:
clf_rfc = RandomForestClassifier(max_depth=None,class_weight='balanced')
clf_rfc.fit(x_train_all, y_train_all)

RandomForestClassifier(class_weight='balanced')

In [27]:
compt = 0
for i in range(0,x_test_all.shape[0]):
    if clf_rfc.predict([x_test_all[i]])[0] - y_test_all[i] == 0.0:
        compt +=1
print(compt)

print(100*compt/x_test_all.shape[0],"% accuracy")

9811
38.6625157629256 % accuracy


In [28]:
#-------------------- svm rbf increment C -----------------
c_values = np.array([0.1 , 0.5 , 1 , 5, 10,15, 20, 50,100,150,300,500,1000])

In [29]:
#----------------- MlP -------------------------
from sklearn.neural_network import MLPClassifier

In [30]:
clf_mlp = MLPClassifier(random_state=0, max_iter=300 , hidden_layer_sizes =(200,1),solver='adam', activation = 'logistic')
clf_mlp.fit(x_train_all, y_train_all)

MLPClassifier(activation='logistic', hidden_layer_sizes=(200, 1), max_iter=300,
              random_state=0)

In [31]:
compt = 0
for i in range(0,x_test_all.shape[0]):
    if clf_mlp.predict([x_test_all[i]])[0] - y_test_all[i] == 0.0:
        compt +=1
print(compt)

print(100*compt/x_test_all.shape[0],"% accuracy")

10313
40.64076292559899 % accuracy


In [32]:
clf_mlp = MLPClassifier(random_state=0, max_iter=300 , hidden_layer_sizes =(400,1),solver='adam', activation = 'logistic')
clf_mlp.fit(x_train_all, y_train_all)

MLPClassifier(activation='logistic', hidden_layer_sizes=(400, 1), max_iter=300,
              random_state=0)

In [33]:
compt = 0
for i in range(0,x_test_all.shape[0]):
    if clf_mlp.predict([x_test_all[i]])[0] - y_test_all[i] == 0.0:
        compt +=1
print(compt)

print(100*compt/x_test_all.shape[0],"% accuracy")

10313
40.64076292559899 % accuracy


In [34]:
clf_mlp = MLPClassifier(random_state=0, max_iter=300 , hidden_layer_sizes =(50,1),solver='adam', activation = 'logistic')
clf_mlp.fit(x_train_all, y_train_all)

MLPClassifier(activation='logistic', hidden_layer_sizes=(50, 1), max_iter=300,
              random_state=0)

In [35]:
compt = 0
for i in range(0,x_test_all.shape[0]):
    if clf_mlp.predict([x_test_all[i]])[0] - y_test_all[i] == 0.0:
        compt +=1
print(compt)

print(100*compt/x_test_all.shape[0],"% accuracy")

10313
40.64076292559899 % accuracy


In [36]:
clf_mlp = MLPClassifier(random_state=0, max_iter=300 , hidden_layer_sizes =(50,1),solver='adam', activation = 'logistic', learning_rate_init=0.01)
clf_mlp.fit(x_train_all, y_train_all)

MLPClassifier(activation='logistic', hidden_layer_sizes=(50, 1),
              learning_rate_init=0.01, max_iter=300, random_state=0)

In [37]:
compt = 0
for i in range(0,x_test_all.shape[0]):
    if clf_mlp.predict([x_test_all[i]])[0] - y_test_all[i] == 0.0:
        compt +=1
print(compt)

print(100*compt/x_test_all.shape[0],"% accuracy")

10316
40.65258511979823 % accuracy


In [38]:
clf_mlp = MLPClassifier(random_state=0, max_iter=300 , hidden_layer_sizes =(50,1),solver='adam', activation = 'logistic', learning_rate_init=0.1)
clf_mlp.fit(x_train_all, y_train_all)

MLPClassifier(activation='logistic', hidden_layer_sizes=(50, 1),
              learning_rate_init=0.1, max_iter=300, random_state=0)

In [39]:
compt = 0
for i in range(0,x_test_all.shape[0]):
    if clf_mlp.predict([x_test_all[i]])[0] - y_test_all[i] == 0.0:
        compt +=1
print(compt)

print(100*compt/x_test_all.shape[0],"% accuracy")

10313
40.64076292559899 % accuracy


In [40]:
clf_mlp = MLPClassifier(random_state=0, max_iter=500 , hidden_layer_sizes =(200,1),solver='adam', activation = 'logistic')
clf_mlp.fit(x_train_all, y_train_all)

MLPClassifier(activation='logistic', hidden_layer_sizes=(200, 1), max_iter=500,
              random_state=0)

In [41]:
compt = 0
for i in range(0,x_test_all.shape[0]):
    if clf_mlp.predict([x_test_all[i]])[0] - y_test_all[i] == 0.0:
        compt +=1
print(compt)

print(100*compt/x_test_all.shape[0],"% accuracy")

10313
40.64076292559899 % accuracy


In [42]:
clf_mlp = MLPClassifier(random_state=0, max_iter=500 , hidden_layer_sizes =(200,1),solver='adam', activation = 'tanh')
clf_mlp.fit(x_train_all, y_train_all)

MLPClassifier(activation='tanh', hidden_layer_sizes=(200, 1), max_iter=500,
              random_state=0)

In [43]:
compt = 0
for i in range(0,x_test_all.shape[0]):
    if clf_mlp.predict([x_test_all[i]])[0] - y_test_all[i] == 0.0:
        compt +=1
print(compt)

print(100*compt/x_test_all.shape[0],"% accuracy")

10313
40.64076292559899 % accuracy


In [44]:
clf_mlp_2 = MLPClassifier(random_state=0, max_iter=500 , hidden_layer_sizes =(100,100),solver='adam', activation = 'logistic', learning_rate_init=0.01)
clf_mlp_2.fit(x_train_all, y_train_all)

MLPClassifier(activation='logistic', hidden_layer_sizes=(100, 100),
              learning_rate_init=0.01, max_iter=500, random_state=0)

In [45]:
compt = 0
for i in range(0,x_test_all.shape[0]):
    if clf_mlp_2.predict([x_test_all[i]])[0] - y_test_all[i] == 0.0:
        compt +=1
print(compt)

print(100*compt/x_test_all.shape[0],"% accuracy")

10335
40.72745901639344 % accuracy


In [46]:
clf_mlp = MLPClassifier(random_state=0, max_iter=500 , hidden_layer_sizes =(200),solver='adam', activation = 'tanh')
clf_mlp.fit(x_train_all, y_train_all)

MLPClassifier(activation='tanh', hidden_layer_sizes=200, max_iter=500,
              random_state=0)

In [47]:
compt = 0
for i in range(0,x_test_all.shape[0]):
    if clf_mlp.predict([x_test_all[i]])[0] - y_test_all[i] == 0.0:
        compt +=1
print(compt)

print(100*compt/x_test_all.shape[0],"% accuracy")

10307
40.617118537200504 % accuracy


In [48]:
clf = svm.SVC(gamma='auto',kernel='rbf',class_weight='balanced',C=1.0, degree=3 ,max_iter= -1)
clf.fit(x_train_all, y_train_all)

SVC(class_weight='balanced', gamma='auto')

In [49]:
compt = 0
for i in range(0,x_test_all.shape[0]):
    if clf.predict([x_test_all[i]])[0] - y_test_all[i] == 0.0:
        compt +=1
print(compt)

print(100*compt/x_test_all.shape[0],"% accuracy")

1146
4.516078184110971 % accuracy


In [50]:
clf_linear = svm.SVC(gamma='auto',kernel='linear',class_weight='balanced',C=1.0, degree=3 ,max_iter= -1)
clf_linear.fit(x_train_all, y_train_all)

SVC(class_weight='balanced', gamma='auto', kernel='linear')

In [51]:
compt = 0
for i in range(0,x_test_all.shape[0]):
    if clf_linear.predict([x_test_all[i]])[0] - y_test_all[i] == 0.0:
        compt +=1
print(compt)

print(100*compt/x_test_all.shape[0],"% accuracy")

1833
7.223360655737705 % accuracy
